<a href="https://colab.research.google.com/github/stinkymatt/pub/blob/master/District13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataStax Ship Team Hackathon
Team: District 13

This notebook connects to an Astra database and pulls data from it.

It uses the Cassandra Python driver, so it needs a secure-connect-bundle.
It can pull that from the GCP Secret Manager, or you can upload a secure-connect-bundle in the Filemanager on the left. Name it **scb.zip**.

You'll also need a client_id and secret for a read-only role. If you downloaded the GeneratedToken.csv file from Astra, you can either upload that, or fill in the field values below.


In [ ]:

#Run me first to install the dependencies
!pip install google-cloud-secret-manager
!pip install astrapy
!pip install cassandra-driver


Below are the settings for our database that are NOT sensitive. 

In [53]:
#config
ASTRA_DB_ID='21107608-ec56-472e-90cf-95ac419e5d7c' #could make an API call to get these //TODO
ASTRA_DB_REGION='us-east-1'
ASTRA_DB_KEYSPACE='district13'
proj_name='gcp-product-401-dev'
secret_name='dist13org'
path_to_secret = f"projects/{proj_name}/secrets/{secret_name}/versions/latest"

You do not need to run the next section if you have uploaded an Astra Secure Connect Bundle named "scb.zip"

If you do run the next section, an authentication step will prompt for an auth token. Click the link and follow the instructions, then paste the provided token back into the empty text field in the notebook output area below the cell.

In [23]:
#NOTE: You can also just upload the secure connect bundle by clicking the folder
#icon on the far left sidebar. Name the file scb.zip

#Get an admin secret from the GCP secret store
#Download the secure connect bundle


from google.cloud import secretmanager
from google.colab import auth
#This line will start a dialog below, follow instructions to authenticate
auth.authenticate_user() 
#!gcloud config set project 'gcp-product-401-dev'

#This uses the GCP cloud API to fetch a secret
secret_store = secretmanager.SecretManagerServiceClient()
resp = secret_store.access_secret_version(request={"name": path_to_secret})
mysecret = resp.payload.data.decode('UTF-8')
ASTRA_DB_APPLICATION_TOKEN=mysecret

#Astra devops API to fetch secure connect bundle
from astrapy.client import create_astra_client

astra_client = create_astra_client(astra_database_id=ASTRA_DB_ID,
                                   astra_database_region=ASTRA_DB_REGION,
                                   astra_application_token=ASTRA_DB_APPLICATION_TOKEN)
zipUrl = astra_client.ops.get_secure_bundle(database=ASTRA_DB_ID)

import urllib
urllib.request.urlretrieve(zipUrl['downloadURL'], 'scb.zip')

adc.json  sample_data  scb.zip


Below, you can manually enter values for the client_id and client_secret that Astra provided. If you have NOT uploaded the GeneratedToken.csv file in the Filemanager on the left, then fill these fields out. You can use either. If the file is present, it will take precedence.

In [46]:
#Enter values for the client ID and client Secret
# OR 
#Upload the GeneratedToken.csv file with the file browser at left
client_id_input = "uCmtnzAThNOEfxzavDiqMDUa" #@param {type: "string"}
client_secret_input = "-3erG0ZYJSL_Q_abhdvIYF-QAsR,dczbGcyp0-uO8eqsejB7rk7QtHoAJUZvEvLZtUn4c9XFBReH63PU0LZJ40Z+hc18YGQ_689rJph1YPmhlh8fx5F2PHKoDPRd3eH_" #@param {type:"string"}
#!cat GeneratedToken.csv

In [54]:
#You can upload an AstraCreds.csv file in the file explorer on the left
#Otherwise you will be prompted for client_id and secret
import csv
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

#If there is no AstraToken.csv in working dir, then prompt user
def get_auth_info_from_creds():
  """
  Collect information about auth id and secred from AstraToken file
  if such file is presentin the gdrive "Colab Notebooks" folder.
  In other case it will return some hardcoded defaults.
  """
  try:
    auth_info = None
    with open('GeneratedToken.csv', newline='') as csvfile: 
      reader = csv.reader(csvfile, delimiter=',', quotechar='"')
      auth_info = [x for x in reader][1]  # 2nd line
    print("returning auth info from GeneratedToken.csv file")
    id = auth_info[0]
    secret = auth_info[1]
    token = auth_info[2]
    return id, secret, token
  except FileNotFoundError:
    print("GeneratedToken.csv not found in the gdrive, set creds manually in fields above.")
    return client_id_input, client_secret_input, None

def connect_to_astra():
  cloud_config= {'secure_connect_bundle':'scb.zip'}
  auth_prov = PlainTextAuthProvider(auth_id, auth_secret)
  cluster = Cluster(cloud=cloud_config, auth_provider=auth_prov)
  return cluster.connect()

auth_id, auth_secret, _ = get_auth_info_from_creds()
session = connect_to_astra()
electricity_data_2020 = session.execute("SELECT * FROM district13.electricity WHERE year=2017").all()
#print("Found {} countries for 2020".format(len(electricity_data_2020)))
electricity_data_2020

returning auth info from AstraCreds file
Found 210 countries for 2020


[Row(iso_code='AGO', year=2017, biofuel_electricity=Decimal('0.2'), carbon_intensity_elec=None, coal_electricity=Decimal('0.0'), country='Angola', electricity_generation=Decimal('10.676'), fossil_electricity=Decimal('2.882'), gas_electricity=Decimal('1.569'), hydro_electricity=Decimal('7.576'), nuclear_electricity=Decimal('0.0'), oil_electricity=Decimal('1.313'), other_renewable_electricity=Decimal('0.2'), other_renewable_exc_biofuel_electricity=Decimal('0.0'), renewables_electricity=Decimal('7.794'), solar_electricity=Decimal('0.018'), wind_electricity=Decimal('0.0')),
 Row(iso_code='VNM', year=2017, biofuel_electricity=Decimal('0.076'), carbon_intensity_elec=None, coal_electricity=Decimal('62.688'), country='Vietnam', electricity_generation=Decimal('191.222'), fossil_electricity=Decimal('102.721'), gas_electricity=Decimal('39.576'), hydro_electricity=Decimal('88.092'), nuclear_electricity=Decimal('0.0'), oil_electricity=Decimal('0.457'), other_renewable_electricity=Decimal('0.076'), 